In [1]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import sys
import numpy as np
import string
import operator
import csv
import urllib2
import json
from collections import defaultdict
import StringIO
import random
from multiprocessing import Pool

In [2]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "gov"
method = "lr" # hal, mi, assoc, assocMi, assocHal, assoc2
simMeasure = "" # mi, cnet
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfgInFileName =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfgOutFileName =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("colIndexDir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
methodGraphsFileName = []
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method.translate(None, string.digits) in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method.translate(None, string.digits), knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
dumpindexStatementFilename = os.path.join("/", "tmp", "sbsb", "statement" + "_" + collection + "_" + method + "_" + knowledgGraph )
print("dumpindexStatementFilename =", dumpindexStatementFilename)
occuranceCountFilename = os.path.join(projectDir, "occuranceCount", "occuranceCount")
print("occuranceCountFilename =", occuranceCountFilename)
countsResultsFile = os.path.join(projectDir,"occuranceCount","results",collection+".txt")
print("countsResultsFile =", countsResultsFile)
conceptnet5RelAllFilename ="/scratch/saeid/data/conceptnet5_simp.csv"
print("conceptnet5RelAllFilename =", conceptnet5RelAllFilename)
cfgTmpOutFileName = cfgOutFileName + ".tmp"
print("cfgTmpOutFileName =", cfgTmpOutFileName)
docIdNameMapFileName = os.path.join(projectDir, "occuranceCount", "results", collection + "_docIdNameMap.txt")
print("docIdNameMapFileName =", docIdNameMapFileName)
#dumpindexResFilename = os.path.join(projectDir,"occuranceCount","results",collection +"_dumpindexRes"+".txt")
#print("dumpindexResFilename =", dumpindexResFilename)
#expansionCount = 25

cfgInFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/gov/queryGov
cfgOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/gov/lr/cnet/indriRunQuery.cfg
colIndexDir = /scratch/index/indri_5_7/gov
knowledgGraph index Dir = /scratch/index/indri_5_7/gov
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/gov/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/lr/gov/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/lr/gov/cnet/indriRunQuery.evals
dumpindexStatementFilename = /tmp/sbsb/statement_gov_lr_cnet
occuranceCountFilename = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/occuranceCount
countsResultsFile = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/gov.txt
conceptnet5RelAllFilename = /scratch/saeid/data/conceptnet5_simp.csv
cfgTmpOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/gov/lr/cnet/indriRunQuery.cfg.tmp
docIdNameMapFileName = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/gov_

In [3]:
# generate docIdNameMapFileName
cmd = ' '.join([occuranceCountFilename, colIndexDir, "dm", ">", docIdNameMapFileName])
print("cmd =", cmd)
res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

cmd = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/occuranceCount /scratch/index/indri_5_7/gov dm > /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/gov_docIdNameMap.txt


In [4]:
with open(docIdNameMapFileName, 'r') as f:
    reader = list(csv.reader(f, delimiter = ' '))
    print(reader[0])
    docIdNameMap = {i[1]:i[0] for i in reader}
    docNameIdMap = {i[0]:i[1] for i in reader}
for i, (k, v) in enumerate(docIdNameMap.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

['1', 'G45-42-0000000']
G27-29-4040585 	 1220122
G06-40-3131996 	 281807
G32-16-0767517 	 336566
G38-20-1081477 	 35031
G09-37-0269135 	 606264
G35-90-1409165 	 1000554
G45-00-1993329 	 6030
G02-14-0310925 	 409727
G24-75-3379489 	 482494
G41-54-1115839 	 613900
G08-45-4056636 	 443549
G07-57-3916819 	 649375
G18-64-1809424 	 382013
G01-53-1599922 	 56902
G09-45-1860255 	 607278
G10-84-0819779 	 126372
G07-75-1332741 	 640405
G41-16-2022415 	 612570
G28-26-3010045 	 735202
G40-48-0701479 	 932948
G10-12-3054008 	 136841
G27-23-2273389 	 1206944


In [5]:
count_history = dict()
if os.path.isfile(countsResultsFile):
    with open(countsResultsFile, 'r') as f:
        reader = csv.reader(f, delimiter = "\t")
        count_history = {k:int(float(v)) for k,v in list(reader)}
    print("size of count_history =", len(count_history))
    for i, (k, v) in enumerate(count_history.iteritems()):
        print (k, '\t', v)
        if (i>20):
            break

In [6]:
"""
dumpindex_colIndexDir_hist = dict()
with open(dumpindexResFilename, 'r') as f:
    reader = csv.reader(f, delimiter = "\t")
    dumpindex_colIndexDir_hist = {k:v for k,v in list(reader)}
print("size of count_history =", len(count_history))
for i, (k, v) in enumerate(count_history.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break
"""

'\ndumpindex_colIndexDir_hist = dict()\nwith open(dumpindexResFilename, \'r\') as f:\n    reader = csv.reader(f, delimiter = "\t")\n    dumpindex_colIndexDir_hist = {k:v for k,v in list(reader)}\nprint("size of count_history =", len(count_history))\nfor i, (k, v) in enumerate(count_history.iteritems()):\n    print (k, \'\t\', v)\n    if (i>20):\n        break\n'

In [7]:
conceptnet5RelAll = defaultdict(dict)
conceptnet5RelAllInv = defaultdict(dict)
with open(conceptnet5RelAllFilename, 'r') as f:
    reader = csv.reader(f, delimiter = ",")
    for line in list(reader):
        if line[0].strip() != "" and line[1].strip() != "" and line[2].strip() != "":
            if all(c in string.printable for c in line[0]) and all(c in string.printable for c in line[1]) and all(c in string.printable for c in line[2]):
                conceptnet5RelAll[line[1].strip()][line[2].strip()] = line[0].strip()
                conceptnet5RelAllInv[line[2].strip()][line[1].strip()] = line[0].strip()
print("size of conceptnet5RelAll =", len(conceptnet5RelAll))
for i, (k1, k2v) in enumerate(conceptnet5RelAll.iteritems()):
    for j, (k2, v) in enumerate(k2v.iteritems()):
        if (i>3 or j>3):
            break
        print (k1, '\t', k2, '\t', v)
        

size of conceptnet5RelAll = 2965684
tripolitan 	 tripoline 	 Synonym
tripolitan 	 tripoli 	 RelatedTo
age of nemesis 	 band 	 IsA
age of nemesis 	 organisation 	 InstanceOf
age of nemesis 	 progressive rock 	 dbpedia/genre
age of nemesis 	 progressive metal 	 dbpedia/genre
joseph john annabring 	 person 	 InstanceOf
british rail class 438 	 mean of transportation 	 InstanceOf
british rail class 438 	 train 	 InstanceOf


In [8]:
def splitStemText(text):
    stemmedWords = []
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    if method in {"assocRestful", "lr"}:
        for text_ in text.split(' '):
            text = text_.replace(" ", "_")
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            text = os.path.basename(data_j[u'uri'])
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            #print("data_j =", data_j)
            #print("data_j[u'uri'] =", data_j[u'uri'])
            stemmedWords += [str(os.path.basename(data_j[u'uri']))]
        #print("stemmedWords =", stemmedWords)
        return (stemmedWords)
    else:
        words = text.split()
        for w in words:
            w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
            w = re.sub('\'','', w) # remove apostrophe
            cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
            #print("cmd =", cmd)
            stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
        return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [9]:
def analyseQueries():
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8"}:
            tags = soup.find_all(['top'])
            #print(tags)
        
        origWordsAll = dict()
        for tag in tags:
            
            if collection in {"aquaint", "gov"}:
                docno = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno = result.group(1).strip()
            
            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
            origWordsAll[docno] = origWords
            
    return (origWordsAll)

In [10]:
dumpindex_colIndexDir_hist = dict()

In [12]:
def dumpindex_colIndexDir(cmd2):
    if ' '.join(cmd2) in dumpindex_colIndexDir_hist:
        #print("exists!")
        return dumpindex_colIndexDir_hist[' '.join(cmd2)]
    else:
        #print("do not exists!")
        cmd = ' '.join(["dumpindex", colIndexDir] + cmd2)
        #print("cmd =", cmd)
        res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        dumpindex_colIndexDir_hist[' '.join(cmd2)] = res
        #with open(cmdResFilename, "a") as f:
        #    f.write(' '.join(cmd2), "\t", res) 
        return res
"""
print("docIdNameMap[\"FT944-15972\"] =", docIdNameMap["FT944-15972"])
print("cmd =", ' '.join(["dv", docIdNameMap["FT944-15972"], "|", "awk", "\'END{print $1}\'"]))
print(dumpindex_colIndexDir(["dv", docIdNameMap["FT944-15972"], "|", "awk", "\'END{print $1}\'"]))
"""
cmd2 = ["s", "|", "awk", "\'NR==2{print $2}\'" ]
colDocumentCount = int(np.float(dumpindex_colIndexDir(cmd2)))
print("colDocumentCount =", colDocumentCount)


colDocumentCount = 1247753


In [13]:
indriRunQuery_hist = dict()

In [14]:
def indriRunQuery_(cmd2):
    if ' '.join(cmd2) in indriRunQuery_hist:
        #print("exists!")
        return indriRunQuery_hist[' '.join(cmd2)]
    else:
        #print("do not exists!")
        cmd = ' '.join(["IndriRunQuery"] + cmd2)
        #print("cmd =", cmd)
        res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        indriRunQuery_hist[' '.join(cmd2)] = res
        #with open(cmdResFilename, "a") as f:
        #    f.write(' '.join(cmd2), "\t", res) 
        return res


In [15]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genMultSimpQuery(argss):
    #    #print("intCoeff =", intCoeff)
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
        #print(tags)

    for args in argss:
        #print("args =", args)
        (docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights) = args
        #print("(...) =", docno, origWords_, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        doc_tag = soupNew.new_tag("query")

        docno_tag = soupNew.new_tag("number")
        docno_tag.string = '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')])
        doc_tag.append(docno_tag)

        text_tag = soupNew.new_tag("text")
        text_tag.string = "#weight(\n" 

        text_tag.string += str(intCoeff0) + " #combine(" 
        for ow in set(origWords_):
            ow = regex.sub(' ', ow)
            text_tag.string += ow + " "
        text_tag.string += ")\n"

        if len(relW)>0: 
            relText_string1 = regex.sub(' ', relW)
            text_tag.string += str(intCoeff1) + " #combine(" 
            #print("relText_string1 =", relText_string1)
            text_tag.string += relText_string1.encode('utf-8')
            text_tag.string += ")\n"

        text_tag.string += ") "

        doc_tag.append(text_tag)

        parameters_tag.append(doc_tag)
        #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "10"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    #print(soupNew.prettify())
    #print("cfgTmpOutFileName =", cfgTmpOutFileName)
    with open( cfgTmpOutFileName, 'a+') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)

In [16]:
def weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):
    
    indriRunQuery_hist = [(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)]
    return indriRunQuery_hist


"""
indriRunQuery_hist = []
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
indriRunQuery_hist += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)


docno = "406"
origWords_ = ["parkinson's", 'disease']
ow = 'disease'
relW = "human illness"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
indriRunQuery_hist += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
print("indriRunQuery_hist =", indriRunQuery_hist)
"""

indriRunQuery_hist = [('407', ['poach', 'wildlife', 'preserve'], 'illegally', 0.7, 0.3, 1600, '/home/fj9124/projects/ir/seq_kb_ir/configs/gov/lr/cnet/indriRunQuery.cfg.tmp'), ('406', ["parkinson's", 'disease'], 'human illness', 0.7, 0.3, 1600, '/home/fj9124/projects/ir/seq_kb_ir/configs/gov/lr/cnet/indriRunQuery.cfg.tmp')]


In [17]:
indriRunQuery_hist_= dict()

In [18]:
def runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_):
    with open( cfgTmpOutFileName, 'w+') as outFile:
        outFile.write("")
    args = []
    #print("indriRunQuery_hist =", indriRunQuery_hist)
    for (docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName) in indriRunQuery_hist:
        #print("cfgTmpOutFileName =", cfgTmpOutFileName)
        if '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')]) not in indriRunQuery_hist_:
            args += [(docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)]
    #print("args =", args)
    if len(args)>0:
        genMultSimpQuery(args)
    cmd = ' '.join(["IndriRunQuery", cfgTmpOutFileName])
    #print("cmd =", cmd)
    indriRun = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #print("indriRun =", indriRun)
    indriRunQuery_hist_tmp = [[j for c, j in enumerate(i.split(' ')) if c in {0, 2, 4}] for i in indriRun.split('\n') if len(i)>0]
    #indriRunQuery_hist_ = dict()
    for i in indriRunQuery_hist_tmp:
        if i[0] in indriRunQuery_hist_:
            indriRunQuery_hist_[i[0]] += [[i[1], i[2]]]
        else:
            indriRunQuery_hist_[i[0]] = [[i[1], i[2]]]
    #indriRunQuery_hist_ = list(set(indriRunQuery_hist_))
    return indriRunQuery_hist_
    #dumpindex_colIndexDir_hist
    #with open
"""
indriRunQuery_hist_ = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_)
for j, (k, v) in enumerate(indriRunQuery_hist_.iteritems()):
    if j > 2:
        break
    print("indriRunQuery_hist_ =", k,v)
"""

indriRunQuery_hist_ = 407_poach_wildlife_preserve_illegally [['G45-38-3536095', '-6.66347'], ['G04-02-1106155', '-6.67768'], ['G03-46-0301434', '-6.77493'], ['G24-60-0056267', '-6.81725'], ['G09-39-3725788', '-6.92418'], ['G15-92-1917296', '-6.99406'], ['G44-12-0950487', '-7.00948'], ['G23-38-2240802', '-7.0107'], ['G18-46-2121642', '-7.03611'], ['G38-71-1874175', '-7.13992']]
indriRunQuery_hist_ = 406_parkinson's_disease_human_illness [['G05-34-1201294', '-5.46075'], ['G45-59-1257647', '-5.59158'], ['G43-49-0092446', '-5.59317'], ['G26-66-3314692', '-5.59317'], ['G14-63-1455568', '-5.59398'], ['G11-10-1472958', '-5.59422'], ['G04-61-2773256', '-5.61366'], ['G10-28-1937180', '-5.61366'], ['G45-24-2426686', '-5.61373'], ['G14-79-3429747', '-5.63501']]


In [19]:
def weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_colIndexDir_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):

    cmd2 = ("ef", "#od4(" + regex.sub(' ', relW) + ")")
    if cmd2 not in dumpindex_colIndexDir_hist:
        dumpindex_colIndexDir_hist.add(cmd2)
    
    #print(indriRunQuery_hist_)
    #print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    #print("topTDocs =", topTDocs)
    
    for documentName in topTDocs:
        cmd2 = ("dcf", docIdNameMap[documentName])
        if cmd2 not in dumpindex_colIndexDir_hist:
            dumpindex_colIndexDir_hist.add(cmd2)

    for ow in origWords_:
        cmd2 = ("ef", "#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " ))")
        if cmd2 not in dumpindex_colIndexDir_hist:
            dumpindex_colIndexDir_hist.add(cmd2)
        
    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                cmd2 = ( "ef", "#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " ))")
                if cmd2 not in dumpindex_colIndexDir_hist:
                    dumpindex_colIndexDir_hist.add(cmd2)
    return dumpindex_colIndexDir_hist
"""
dumpindex_colIndexDir_hist = set()
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
dumpindex_colIndexDir_hist |= weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_colIndexDir_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)

docno = "406"
origWords_ = ["parkinson's", 'disease']
ow = 'disease'
relW = "human illness"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
#print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)

dumpindex_colIndexDir_hist |= weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_colIndexDir_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
print("dumpindex_colIndexDir_hist =", dumpindex_colIndexDir_hist)
"""

dumpindex_colIndexDir_hist = set([('ef', '#uw(#4( preserve ) #4( poach ) #4( illegally ))'), ('dcf', '1028138'), ('dcf', '769001'), ('dcf', '193261'), ('dcf', '584215'), ('ef', '#uw(#4( poach ) #4( illegally ))'), ('dcf', '1095356'), ('dcf', '187595'), ('dcf', '217438'), ('dcf', '514538'), ('ef', '#uw(#4( preserve ) #4( wildlife ) #4( illegally ))'), ('dcf', '383751'), ('dcf', '99718'), ('dcf', '1865'), ('ef', '#uw(#4( parkinson s ) #4( human illness ))'), ('ef', '#od4(illegally)'), ('ef', '#uw(#4( preserve ) #4( illegally ))'), ('dcf', '500452'), ('ef', '#uw(#4( wildlife ) #4( illegally ))'), ('dcf', '571915'), ('dcf', '1128'), ('ef', '#uw(#4( disease ) #4( human illness ))'), ('dcf', '49170'), ('dcf', '137533'), ('dcf', '697095'), ('dcf', '564102'), ('ef', '#od4(human illness)'), ('dcf', '13519'), ('ef', '#uw(#4( disease ) #4( parkinson s ) #4( human illness ))'), ('ef', '#uw(#4( wildlife ) #4( poach ) #4( illegally ))'), ('dcf', '146352')])


In [20]:
dumpindex_hist_ = dict()

In [21]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def runCommandsinHist_d(dumpindex_colIndexDir_hist_list, dumpindex_hist_):
    args = []
    dumpindex_colIndexDir_hist_dict = dict()
    tmp = set()
    #print("len(dumpindex_colIndexDir_hist_list) =", len(dumpindex_colIndexDir_hist_list))
    for i in dumpindex_colIndexDir_hist_list:
        #print("i =", i)
        if i[1] not in dumpindex_hist_.get(i[0], dict()):
            #print("dumpindex_hist_.get(i[0], dict()).keys() =", dumpindex_hist_.get(i[0], dict()).keys())
            if i[1] in tmp:
                continue
            tmp.add(i[1])
            #print("i[1] =", i[1])
            #print(i, "not exists!")
            if i[0] in dumpindex_colIndexDir_hist_dict:
                dumpindex_colIndexDir_hist_dict[i[0]] += [[re.sub(ur"[^\w\d#()\s]+",' ',j) for j in i[1:]]]
            else:
                dumpindex_colIndexDir_hist_dict[i[0]] =  [[re.sub(ur"[^\w\d#()\s]+",' ',j) for j in i[1:]]]
    #print(dumpindex_colIndexDir_hist_dict)
        #else:
        #    print("exists!")
    #print("dumpindex_colIndexDir_hist_ =", dumpindex_colIndexDir_hist_)
    #print("len(dumpindex_colIndexDir_hist_.get('ef', dict()) =", len(dumpindex_colIndexDir_hist_.get('ef', dict())))
    #dumpindex_hist_ = dict()
    for k, v in dumpindex_colIndexDir_hist_dict.iteritems():
        fileName = dumpindexStatementFilename + "_" + k + ".tmp"
        with open(fileName, 'w') as f:
            for i in v:
                f.write(' '.join(i) + '\n')
        cmd = ' '.join([occuranceCountFilename, colIndexDir, k, fileName])
        #print("cmd =", cmd)
        dumpindex_hist_tmp = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        dumpindex_hist_tmp = dict([tuple(j for j in i.split(':')) for i in dumpindex_hist_tmp.split("\n") if len(i)>0])
        dumpindex_hist_tmp = {k1:v1.split(',') for k1,v1 in dumpindex_hist_tmp.iteritems()}
        dumpindex_hist_tmp = {k1:[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_tmp.iteritems()}
        #dumpindex_hist_tmp = {re.sub(ur"[^\w\d#()\s]+",' ',k1):[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_tmp.iteritems()}
        #dumpindex_tmp = [i for i in dumpindex_tmp.split("\n")]
        #print("dumpindex_tmp =", dumpindex_tmp)
        #dumpindex_hist_[k] = dumpindex_hist_tmp
        dumpindex_hist_[k] = dict(dumpindex_hist_.get(k, dict()).items() + dumpindex_hist_tmp.items())
        #print("dumpindex_hist_[k].keys() =", dumpindex_hist_[k].keys())
        #print("dumpindex_hist_tmp =", dumpindex_hist_tmp)
        
    return dumpindex_hist_
"""
dumpindex_hist_ = runCommandsinHist_d(dumpindex_colIndexDir_hist, dumpindex_hist_)
#print("dumpindex_hist_['ef'] =", dumpindex_hist_['ef'])
#print("len(dumpindex_hist_['ef']) =", len(dumpindex_hist_['ef']))
"""

In [22]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def documentLength(documentName):
    #cmd = ' '.join(["dumpindex", colIndexDir, "dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"])
    #cmd2 = ["dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"]
    #print("cmd =", cmd)
    #documentLength_ = int(dumpindex_colIndexDir(cmd2))
    documentLength_ = int(dumpindex_hist_['dcf'][documentName][0])
    return documentLength_

#print("docIdNameMap[\"FT944-15973\"] = ", docIdNameMap["FT944-15973"])
#print("documentLength(\"FT944-15973\") =", documentLength("FT944-15973"))

def weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights):
    
    features = dict()
    
    ##########
    #cfgTmpOutFileName = os.path.join("/","tmp","sbsb",str(random.randint(1,1e6))+"_tmp.cfg")
    ##########
    #(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    #genOneSimpQuery(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    
    #indriRun = indriRunQuery_([cfgTmpOutFileName])
    #subprocess.Popen(' '.join(["rm", cfgTmpOutFileName]), shell=True, stdout=subprocess.PIPE).stdout.read()
    #indriRun = [i.split(' ') for i in indriRun.split('\n')]
    #indriRun = [i for i in indriRun if len(i)>1] # remove empty arrays
    #print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    #print("indriRunQuery_hist_1 =", indriRunQuery_hist_1)
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    
    #topTDocScores_d = {i[2]:np.float(i[4]) for i in indriRun}
    topTDocScores_d = {i[0]:np.float(i[1]) for i in indriRunQuery_hist_1}
    
    #scores = [ np.float(i[4]) for i in indriRun]
    #topTDocs = [i[2] for i in indriRun]

    #print("indriRun =", indriRun)
    #topTDocScore = np.sum([ np.float(i[4]) for i in indriRun])
    #expTDocScore = np.float(indriRun[0][4])
    expTDocScore = np.float(indriRunQuery_hist_1[0][1])
    features["expTDocScore"] = expTDocScore
    #print("expTDocScore =", features["expTDocScore"])
    
    #cmd =' '.join(["dumpindex", colIndexDir, "e", "\"#od4(" + relW + ")\""])
    #cmd2 = ["e", "\"#od4(" + relW + ")\""]
    #print("cmd =", cmd)
    #relWDocuments = dumpindex_colIndexDir(cmd2)
    relWDocuments = dumpindex_hist_['ef']["#od4(" + regex.sub(' ', relW) + ")"]
    #relWDocuments = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #relWDocuments = [i.split(' ') for i in relWDocuments.split('\n')]
    #print("relWDocuments =", relWDocuments)
    #relWDocuments = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(relWDocuments) if len(i)==4 and j>0] # remove empty arrays
    #print("relWDocuments =", relWDocuments)
    
    #relWTopDocuments = [i for i in relWDocuments if i[0] in topTDocs]
    #relWTopDocuments = [i for i in relWDocuments if i in topTDocs]
    relWTopDocuments = [[i, topTDocScores_d[i]] for i in relWDocuments if i in topTDocs] # topdocs and their scores that contain relW
    #print("relWTopDocuments =", relWTopDocuments)
    #numerator = np.sum([int(i[1]) for i in relWTopDocuments]) # term count in the collection
    numerator = len(relWDocuments) # relW term count in the collection
    #print("topTDocs =", topTDocs)
    denumerator = np.sum([documentLength(i) for i in topTDocs]) # number of terms in topDocs
    topTermFrac = numerator / np.float(denumerator)
    #print("numerator, denumerator, topTermFrac =", numerator, denumerator, topTermFrac)
    features["topTermFrac"] = topTermFrac
    
    numCanDocs = len(set([i[0] for i in relWTopDocuments]))
    features["numCanDocs"] = numCanDocs
    #print("numCanDocs =", features["numCanDocs"])
    
    if numCanDocs > 0:
        avgCDocScore = np.sum([ topTDocScores_d[i] for i in set([i[0] for i in relWTopDocuments])])/np.float(numCanDocs)
    else:
        avgCDocScore = 0
    features["avgCDocScore"] = avgCDocScore
    #print("avgCDocScore =", features["avgCDocScore"])

    #for i in set([i[0] for i in relWTopDocuments]):
    #    print(i, topTDocScores_d[i])
    
    l = [ topTDocScores_d[i] for i in set([i[0] for i in relWTopDocuments])]
    if len(l)>0:
        maxCDocScore = np.max(l)
    else:
        maxCDocScore = 0
    features["maxCDocScore"] = maxCDocScore
    #print("maxCDocScore =", features["maxCDocScore"])
    
    if len(relWDocuments)>0:
        conIdf = np.log(colDocumentCount / len(relWDocuments))
    else:
        conIdf = np.log(colDocumentCount / 1)
    features["conIdf"] = conIdf
    #print("conIdf =", features["conIdf"])
    
    coocurDocuments = dict()
    coocurDocumentsTop = dict()
    for ow in origWords_:
        #cmd = ' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""])
        #cmd2 = ["e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""]
        #print("cmd =", cmd)
        #coocurDocuments_ = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
        #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
        coocurDocuments[ow] = [i for i in dumpindex_hist_['ef']["#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
        #coocurDocuments[ow] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
        #print("topTDocs =", topTDocs)
        coocurDocumentsTop[ow] = [i for i in coocurDocuments[ow] if i in topTDocs ] # remove empty arrays

    #for k, v in coocurDocuments.iteritems():
    #    print(k, len(v))
        
    #for v in coocurDocumentsTop.values():
    #    print(v)
    #    print(len(set(v)))
        
    avgColCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColCor"] = avgColCor
    #print("avgColCor =", features["avgColCor"])
    
    maxColCor = np.max([len(set(i)) for i in coocurDocuments.values()])
    features["maxColCor"] = maxColCor
    #print("maxColCor =", features["maxColCor"])
    
    avgTopColCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColCor"] = avgTopColCor
    #print("avgTopColCor =", features["avgTopColCor"])
    
    l = [len(set(i)) for i in coocurDocumentsTop.values()]
    if len(l)>0:
        maxTopColCor = np.max(l)
    else:
        maxTopColCor = 0
    features["maxTopColCor"] = maxTopColCor
    #print("maxTopColCor =", features["maxTopColCor"])

    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                #coocurDocuments_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]), shell=True, stdout=subprocess.PIPE).stdout.read()
                #cmd2 = [ "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]
                #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
                #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
                coocurDocuments[ow1+ow2] = [i for i in dumpindex_hist_['ef']["#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
                #coocurDocuments[ow1+ow2] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
                coocurDocumentsTop[ow1+ow2] = [i for i in coocurDocuments[ow1+ow2] if i in topTDocs ] # remove empty arrays

    avgColPCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColPCor"] = avgColPCor
    #print("avgColPCor =", features["avgColPCor"])

    l = [len(set(i)) for i in coocurDocuments.values()]
    if len(l)>0:
        maxColPCor = np.max(l)
    else:
        maxColPCor = 0
    features["maxColPCor"] = maxColPCor
    #print("maxColPCor =", features["maxColPCor"])
    
    avgTopColPCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColPCor"] = avgTopColPCor
    #print("avgTopColPCor =", features["avgTopColPCor"])

    l = [len(set(i)) for i in coocurDocumentsTop.values()] 
    if len(l)>0:
        maxTopColPCor = np.max(l)
    else:
        maxTopColPCor = 0
    features["maxTopColPCor"] = maxTopColPCor
    #print("maxTopColPCor =", features["maxTopColPCor"])
    
    #print("docno, origWords_, relW =", docno, origWords_, relW)
    #print("features =", features)
    
    relWg = 0
    for feature, weight in featureWeights.iteritems():
        relWg += features[feature] * weight
    
    #print("features.keys() =", features.keys())
    return relWg
"""
docno = "407"
origWords_ = ['poach', 'wildlife', 'preserve']
ow = 'poach'
relW = "illegally"
intCoeff0 = 0.7
intCoeff1 = 0.3
intCoeff2 = 0 
dirCoeff = 1600
featureWeights = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
print("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)
"""

'\ndocno = "407"\norigWords_ = [\'poach\', \'wildlife\', \'preserve\']\now = \'poach\'\nrelW = "illegally"\nintCoeff0 = 0.7\nintCoeff1 = 0.3\nintCoeff2 = 0 \ndirCoeff = 1600\nfeatureWeights = {\'maxTopColCor\': 0.1, \'expTDocScore\': 0.1, \'maxColPCor\': 0.1, \'avgCDocScore\': 0.1, \'topTermFrac\': 0.1, \'avgColCor\': 0.1, \'avgTopColCor\': 0.1, \'numCanDocs\': 0.1, \'maxTopColPCor\': 0.1, \'maxCDocScore\': 0.1, \'avgColPCor\': 0.1, \'conIdf\': 0.1, \'avgTopColPCor\': 0.1, \'maxColCor\': 0.1}\nprint("docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights =", docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)\nweightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)\n'

In [23]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords_train(args, indriRunQuery_hist_, dumpindex_hist_):
    #print("origWords =", origWords)
    text_string = ""
    
    count_history, rWords, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights = args
        
    relWords = dict()
    #if method == "relAll":
            
    if method == "lr":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        for ow in origWords_:
            #print("ow =", ow)
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            indriRunQuery_hist = []
            for relW in conceptnet5RelAll[ow].keys():
                indriRunQuery_hist += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            indriRunQuery_hist_ = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_)
            #print("len(indriRunQuery_hist_) =", len(indriRunQuery_hist_))
            
            dumpindex_colIndexDir_hist = set()
            for relW in conceptnet5RelAll[ow].keys():
                dumpindex_colIndexDir_hist |= weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_colIndexDir_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            dumpindex_hist_ = runCommandsinHist_d(dumpindex_colIndexDir_hist, dumpindex_hist_)
            #print("len(dumpindex_hist_['ef']) =", len(dumpindex_hist_['ef']))
            #print("len(dumpindex_hist_['dcf']) =", len(dumpindex_hist_['dcf']))

    return(indriRunQuery_hist_, dumpindex_hist_)

In [24]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(args, indriRunQuery_hist_, dumpindex_hist_):
    #print("origWords =", origWords)
    text_string = ""
    
    count_history, rWords, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights = args
        
    relWords = dict()
    #if method == "relAll":
        
    if method == "assocRestful":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        url = "http://conceptnet5.media.mit.edu/data/5.4/assoc/list/en/"+','.join(origWords_)
        response = urllib2.urlopen(url)
        data = response.read()
        data_j = json.loads(data)
        relWords = {str(d[0].encode('utf-8')).replace('/c/en/', '').replace('-', ' '):np.float(d[1]) for d in data_j[u'similar'] if '/c/en/' in str(d[0].encode('utf-8'))}
    
    elif method == "lr":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        """
        for ow in origWords_:
            #print("ow =", ow)
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            indriRunQuery_hist = []
            for relW in conceptnet5RelAll[ow].keys():
                indriRunQuery_hist += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            indriRunQuery_hist_ = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_)
            #print("len(indriRunQuery_hist_) =", len(indriRunQuery_hist_))
            
            dumpindex_colIndexDir_hist = set()
            for relW in conceptnet5RelAll[ow].keys():
                dumpindex_colIndexDir_hist |= weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_colIndexDir_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            dumpindex_hist_ = runCommandsinHist_d(dumpindex_colIndexDir_hist, dumpindex_hist_)
            #print("len(dumpindex_hist_['ef']) =", len(dumpindex_hist_['ef']))
            #print("len(dumpindex_hist_['dcf']) =", len(dumpindex_hist_['dcf']))
        """
        for ow in origWords_:
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            for relW in conceptnet5RelAll[ow].keys():
                relWg = weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)
                if relW in relWords:
                    relWords[relW] += relWg
                else:
                    relWords[relW] = relWg  
    
    else:
        count_history = countExpr(count_history, origWords, rWords)
        expressions = set()
        for ow in origWords:
            rWords_ow = rWords.get(ow, [])
            expressions.add("#4( " + ow.translate(None, string.punctuation) + " )")
            if ow.strip() == "":
                continue
            if simMeasure == "mi":
                N_w = countExpr_get_1(count_history, ow)
            for rw_wv in rWords_ow:
                (rw_w, rw_v) = rw_wv.items()[0]
                if rw_w.strip() == "":
                    continue           
                if simMeasure == "mi":
                    N_v = countExpr_get_1(count_history, rw_w)
                    N_wv = countExpr_get_2(count_history, ow, rw_w)
                    rw_v_ = mi_(N_w, N_v, N_wv)
                elif simMeasure == "cnet":
                    rw_v_ = rw_v
                relWords[rw_w] = relWords.get(rw_w, 0) + rw_v_
    
    relWords_sorted = sorted(relWords.items(), key=operator.itemgetter(1), reverse=True)
    #print ("origWords =", origWords)
    #print ("relWords_sorted =", relWords_sorted)
    #counter = 0
    relWords_sel = []
    for counter, (rw_w, rw_v) in enumerate(relWords_sorted):
        if (counter >= expansionCount):
            break
        if  all(c in string.printable for c in rw_w):
            if rw_w not in origWords:
                rw_w = regex.sub(' ', rw_w)
                text_string += rw_w + " "
                relWords_sel += [rw_w]
        #counter += 1
    #print("counter =", counter)
#    return(relWords_sel)
    return(count_history, text_string, relWords_sel)

In [25]:
def getAllRelWords_train(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for c1, (docno, origWords) in enumerate(origWordsAll.iteritems()):
        print(docno, "---", origWords)
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        indriRunQuery_hist_, dumpindex_hist_ = getRelWords_train(args, indriRunQuery_hist_, dumpindex_hist_)
        print(c1, "- len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']) =", len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']))
    
    #print("relText_stringAll =", relText_stringAll)
    return (indriRunQuery_hist_, dumpindex_hist_)

In [26]:
def getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for c1, (docno, origWords) in enumerate(origWordsAll.iteritems()):
        #print(str(c1), "---", docno, "---", origWords)
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_, dumpindex_hist_)
        #print("len(indriRunQuery_hist_), len(dumpindex_hist_['dcf']), len(dumpindex_hist_['ef']) =", len(indriRunQuery_hist_), len(dumpindex_hist_['dcf']), len(dumpindex_hist_['ef']))
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_[0:N1]
        #print(docno, "---", origWords, "---", relWords_sel_)
    relText_stringAll += [relText_string]
    
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords_new in relWords_sel.iteritems():
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_, dumpindex_hist_)
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_
    relText_stringAll += [relText_string]
    
    #print("relText_stringAll =", relText_stringAll)
    return (count_history, relText_stringAll)

In [27]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(cfgOutFileName_, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
            #print(tags)

        for docno, origWords in origWordsAll.iteritems():
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            docno_tag.string = docno
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(\n" 
                    
            text_tag.string += str(intCoeff0) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ")\n"

            if len(relText_stringAll[0])>0:
                if len(relText_stringAll[0][docno]) > 3: 
                    relText_string1 = regex.sub(' ', relText_stringAll[0][docno])
                    text_tag.string += str(intCoeff1) + " #combine(" 
                    #print("relText_string1 =", relText_string1)
                    text_tag.string += relText_string1.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print(len(relText_stringAll[1]))
            if len(relText_stringAll[1])>0:
                if len(relText_stringAll[1][docno]) > 3: 
                    relText_string2 = regex.sub(' ', relText_stringAll[1][docno])
                    text_tag.string += str(intCoeff2) + " #combine(" 
                    #print("relText_string2 =", relText_string2)
                    text_tag.string += relText_string2.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print("origWords =", origWords)
            #print("relWords_sel1 =", relWords_sel1)
            #print("relWords_sel2 =", relWords_sel2)
            
            
            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName_, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)


In [28]:
def precisionCompute_train(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):
    #randNum = random.randint(1,1e9)
    origWordsAll = analyseQueries()
    indriRunQuery_hist_, dumpindex_hist_ = getAllRelWords_train(count_history, rWords, origWordsAll, expansionCounts1[0], N1s[0], expansionCounts2[0], intCoeffs0[0], intCoeffs1[0], intCoeffs2[0], dirCoeffs[0], featureWeights_[0], indriRunQuery_hist_, dumpindex_hist_)
                                    
    return origWordsAll, count_history, indriRunQuery_hist_, dumpindex_hist_

In [29]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 85 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
rWords = dict()
featureWeights_1 = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
featureWeights_l=[featureWeights_1]
origWordsAll, count_history, indriRunQuery_hist_, dumpindex_hist_ = precisionCompute_train(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)


216 --- ['windandsea']
0 - len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']) = 2 11 20
217 --- ['astronaut', 'biography']
1 - len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']) = 86 347 157
214 --- ['local', 'area', 'unemployment', 'statistic', 'home']
2 - len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']) = 1032 15464 2242
215 --- ['nih', 'video', 'cast']
3 - len(indriRunQuery_hist_), len(dumpindex_hist_['ef']), len(dumpindex_hist_['dcf']) = 1242 16906 2816
212 --- ['national', 'mediation', 'board']


KeyboardInterrupt: 

In [ ]:
def precisionCompute(origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):
    #mapPrecs = dict()
    for featureWeights in featureWeights_:
        for N1 in N1s:
            for intCoeff0 in intCoeffs0:
                for intCoeff1 in intCoeffs1:
                    for intCoeff2 in intCoeffs2:
                        for expansionCount1 in expansionCounts1:
                            for expansionCount2 in expansionCounts2:
                                for dirCoeff in dirCoeffs:
                                    
                                    
                                    #res = getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                                    
                                    count_history, relText_stringAll = getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_)
                                    
                                    print("len(relText_stringAll[1]) =", len(relText_stringAll[1]))
                                    ##########
                                    #cfgOutFileName = os.path.join("tmp","sbsb",str(randNum)+".cfg")
                                    #runsFileName = os.path.join("tmp","sbsb",str(randNum)+".run")
                                    #evalsFileName = os.path.join("tmp","sbsb",str(randNum)+".eval")
                                    ##########
                                    genQueries(cfgOutFileName, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                    subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                    subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                    #print ("cmd = ", cmd)
                                    mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    print("featureWeights =", featureWeights)
                                    print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                    #mapPrecs[intCoeff] = mapPrec

In [ ]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 85 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
#N1s = range(1, 30, 1)
#mapPrecs = 
rWords = dict()
featureWeights_l=[]
featureWeights_1 = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
for expTDocScore in [-150, -50, -5, -0.5, 0.5, 5, 50, 150 ]:
    featureWeights_1["expTDocScore"] = expTDocScore
    featureWeights_l += [dict(featureWeights_1)] 
precisionCompute(origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)
